# Data Exploration - Initial Summary

This notebook loads the CSV files from the data folder and provides an initial summary of the datasets before deeper exploration.

In [17]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [18]:
# Set up data directory path
data_dir = Path('../data')
print(f"Data directory: {data_dir.absolute()}")
print(f"\nCSV files found:")
csv_files = list(data_dir.glob('*.csv'))
for file in csv_files:
    print(f"  - {file.name}")

Data directory: c:\Users\easan.s.2024\Documents\GitHub\CS201-G4T6\TopK-Airlines\EDA\..\data

CSV files found:
  - airline.csv
  - airport.csv
  - lounge.csv
  - seat.csv


In [19]:
# Load all CSV files into a dictionary
dataframes = {}
for csv_file in csv_files:
    df_name = csv_file.stem  # Get filename without extension
    dataframes[df_name] = pd.read_csv(csv_file)
    print(f"Loaded: {df_name}")

Loaded: airline
Loaded: airport
Loaded: lounge
Loaded: seat


## Data Cleaning

Clean all datasets by:
1. Dropping columns with >40% null values
2. Dropping rows with null values in columns that have 10-40% null values
3. Dropping rows with null values in columns that have <10% null values

In [20]:
# Clean all datasets
cleaned_dataframes = {}

for name, df in dataframes.items():
    print("=" * 80)
    print(f"CLEANING DATASET: {name}")
    print("=" * 80)
    
    original_shape = df.shape
    print(f"Original shape: {original_shape[0]:,} rows × {original_shape[1]} columns")
    
    # Step 1: Identify and drop columns with >40% null values
    null_percentages = (df.isnull().sum() / len(df)) * 100
    cols_to_drop = null_percentages[null_percentages > 40].index.tolist()
    
    if cols_to_drop:
        print(f"\nDropping {len(cols_to_drop)} columns with >40% null values:")
        for col in cols_to_drop:
            print(f"  - {col}: {null_percentages[col]:.2f}% null")
        df = df.drop(columns=cols_to_drop)
    else:
        print("\nNo columns with >40% null values found.")
    
    # Step 2: Identify columns with 10-40% null values and drop rows with nulls in these columns
    remaining_null_pct = (df.isnull().sum() / len(df)) * 100
    cols_medium_nulls = remaining_null_pct[(remaining_null_pct >= 10) & (remaining_null_pct <= 40)].index.tolist()
    
    if cols_medium_nulls:
        print(f"\nDropping rows with null values in {len(cols_medium_nulls)} columns (10-40% nulls):")
        for col in cols_medium_nulls:
            print(f"  - {col}: {remaining_null_pct[col]:.2f}% null")
        df = df.dropna(subset=cols_medium_nulls)
    else:
        print("\nNo columns with 10-40% null values found.")
    
    # Step 3: Identify columns with <10% null values and drop rows with nulls in these columns
    remaining_null_pct = (df.isnull().sum() / len(df)) * 100
    cols_to_clean = remaining_null_pct[(remaining_null_pct > 0) & (remaining_null_pct < 10)].index.tolist()
    
    if cols_to_clean:
        print(f"\nDropping rows with null values in {len(cols_to_clean)} columns (<10% nulls):")
        for col in cols_to_clean:
            print(f"  - {col}: {remaining_null_pct[col]:.2f}% null")
        df = df.dropna(subset=cols_to_clean)
    else:
        print("\nNo columns with <10% null values found.")
    
    # Store cleaned dataframe
    cleaned_dataframes[name] = df
    
    final_shape = df.shape
    rows_removed = original_shape[0] - final_shape[0]
    cols_removed = original_shape[1] - final_shape[1]
    
    print(f"\nFinal shape: {final_shape[0]:,} rows × {final_shape[1]} columns")
    print(f"Removed: {rows_removed:,} rows ({(rows_removed/original_shape[0]*100):.2f}%) and {cols_removed} columns")
    print(f"Remaining null values: {df.isnull().sum().sum():,}")
    print("\n")

# Replace original dataframes with cleaned versions
dataframes = cleaned_dataframes

print("=" * 80)
print("CLEANING COMPLETE - All datasets updated")
print("=" * 80)

CLEANING DATASET: airline
Original shape: 41,396 rows × 20 columns

Dropping 5 columns with >40% null values:
  - aircraft: 96.91% null
  - type_traveller: 94.26% null
  - route: 94.34% null
  - ground_service_rating: 94.68% null
  - wifi_connectivity_rating: 98.64% null

Dropping rows with null values in 5 columns (10-40% nulls):
  - overall_rating: 10.96% null
  - seat_comfort_rating: 18.58% null
  - cabin_staff_rating: 18.57% null
  - food_beverages_rating: 19.64% null
  - inflight_entertainment_rating: 24.84% null

Dropping rows with null values in 2 columns (<10% nulls):
  - author_country: 0.13% null
  - cabin_flown: 3.60% null

Final shape: 27,284 rows × 15 columns
Removed: 14,112 rows (34.09%) and 5 columns
Remaining null values: 0


CLEANING DATASET: airport
Original shape: 17,721 rows × 20 columns

Dropping 8 columns with >40% null values:
  - experience_airport: 96.35% null
  - date_visit: 96.65% null
  - type_traveller: 96.35% null
  - terminal_seating_rating: 96.69% null
 

## Dataset Summaries

Let's examine each dataset's structure, size, and basic statistics.

In [21]:
# Display summary for each dataset
for name, df in dataframes.items():
    print("=" * 80)
    print(f"Dataset: {name}")
    print("=" * 80)
    print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\nColumns and Data Types:")
    print(df.dtypes)
    print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("No missing values")
    print("\n")

Dataset: airline

Shape: 27284 rows × 15 columns

Columns and Data Types:
airline_name                      object
link                              object
title                             object
author                            object
author_country                    object
date                              object
content                           object
cabin_flown                       object
overall_rating                   float64
seat_comfort_rating              float64
cabin_staff_rating               float64
food_beverages_rating            float64
inflight_entertainment_rating    float64
value_money_rating               float64
recommended                        int64
dtype: object

Memory Usage: 32.58 MB

Missing Values:
No missing values


Dataset: airport

Shape: 10817 rows × 12 columns

Columns and Data Types:
airport_name                    object
link                            object
title                           object
author                          object
author

In [22]:
# Display first few rows and descriptive statistics for each dataset
for name, df in dataframes.items():
    print("=" * 80)
    print(f"Dataset: {name} - First 5 Rows")
    print("=" * 80)
    display(df.head())
    
    print(f"\nDescriptive Statistics for {name}:")
    print("=" * 80)
    display(df.describe(include='all'))
    print("\n\n")

Dataset: airline - First 5 Rows


,airline_name,link,title,author,author_country,date,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,Economy,7.0,4.0,4.0,4.0,0.0,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,Business Class,10.0,4.0,5.0,4.0,1.0,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,Economy,9.0,5.0,5.0,4.0,0.0,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,Business Class,8.0,4.0,4.0,3.0,1.0,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,Economy,4.0,4.0,2.0,1.0,2.0,2.0,0



Descriptive Statistics for airline:


,airline_name,link,title,author,author_country,date,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,value_money_rating,recommended
count,27284,27284,27284,27284,27284,27284,27284,27284,27284.000000,27284.000000,27284.000000,27284.000000,27284.000000,27284.000000,27284.000000
unique,292,292,292,19634,142,956,27265,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,british-airways,/airline-reviews/british-airways,British Airways customer review,Tercon Bojan,United Kingdom,2015-01-19,Rating : 10/10 Cabin Flown Economy Value for M...,Economy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,855,855,855,53,6275,266,3,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.067879,3.259566,3.522944,3.016493,2.553108,3.411413,0.603870
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.216066,1.351689,1.460053,1.515096,1.696009,1.440927,0.489101
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2.000000,2.000000,2.000000,1.000000,2.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,4.000000,4.000000,3.000000,3.000000,4.000000,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,4.000000,5.000000,4.000000,4.000000,5.000000,1.000000





Dataset: airport - First 5 Rows


...

,airline_name,link,title,author,author_country,date,content,lounge_name,airport,lounge_type,overall_rating,comfort_rating,cleanliness_rating,bar_beverages_rating,catering_rating,washrooms_rating,wifi_connectivity_rating,staff_service_rating,recommended
count,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
unique,85,85,846,1090,64,236,1467,521,177,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,british-airways,/lounge-reviews/british-airways,Turkish Airlines Business Class Lounge - Istan...,C Wajsberg,United Kingdom,2013-07-07,We were booked into a United BusinessFirst cab...,EMIRATES BUSINESS CLASS LOUNGE REVIEW,London Heathrow Airport,Business Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,175,175,33,14,392,44,3,58,116,1290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.415646,3.339456,3.665306,3.231293,2.793878,3.055782,3.376190,3.279592,0.456463
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.531836,1.332837,1.259769,1.401973,1.479735,1.460957,1.462247,1.424785,0.498270
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,5.000000,1.000000





Dataset: seat - First 5 Rows


,airline_name,link,title,author,author_country,date,content,aircraft,seat_layout,cabin_flown,overall_rating,seat_legroom_rating,seat_recline_rating,seat_width_rating,aisle_space_rating,viewing_tv_rating,recommended
0,aegean-airlines,/seat-reviews/aegean-airlines,Aegean Airlines customer review,Jay Simpson,United Kingdom,2015-07-20,LHR to Larnaca return. Plane was clean and in ...,A320-200,3x3,Economy,10.0,4,4,4,5,4.0,1
1,aegean-airlines,/seat-reviews/aegean-airlines,Aegean Airlines customer review,Paul Staples,United Kingdom,2013-01-21,For a short haul airline the seats are very go...,AIRBUS A320,3x3,Economy,9.0,4,4,4,4,4.0,1
2,aer-lingus,/seat-reviews/aer-lingus,Aer Lingus customer review,L Pulliam,United States,2015-07-07,The seats are a bit tight but bearable. If you...,A330,2x4x2,Economy,6.0,3,3,3,3,3.0,1
3,aer-lingus,/seat-reviews/aer-lingus,Aer Lingus customer review,D Brose,United States,2010-10-22,Appeared new. Good PTV entertainment. Seats ha...,Airbus A330,2x4x2,Economy,5.0,2,3,3,3,4.0,0
4,aeroflot-russian-airlines,/seat-reviews/aeroflot-russian-airlines,Aeroflot Russian Airlines customer review,Konstantinos Grimpilakos,Greece,2015-08-02,Boeing 737-800 seats from Athens to Moscow are...,Boeing 737-800,3x3,Economy,1.0,1,1,1,2,1.0,1



Descriptive Statistics for seat:


,airline_name,link,title,author,author_country,date,content,aircraft,seat_layout,cabin_flown,overall_rating,seat_legroom_rating,seat_recline_rating,seat_width_rating,aisle_space_rating,viewing_tv_rating,recommended
count,1209,1209,1209,1209,1209,1209,1209,1209,1209,1209,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000
unique,90,90,90,1111,58,274,1203,146,30,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,emirates,/seat-reviews/emirates,Emirates customer review,J Wong,United Kingdom,2014-11-19,AF seats narrow and hard and it was very diffi...,BOEING 747-400,2x4x2,Economy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,84,84,84,6,333,74,2,105,386,1030,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.298594,2.743590,2.642680,2.711332,2.717949,2.874276,0.355666
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.035825,1.442758,1.266493,1.275162,1.340536,1.450876,0.478913
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


## Deep Dive: Multi-Dataset Analysis

Let's conduct a thorough analysis of all datasets to understand their properties, distributions, and identify potential computational challenges.

In [23]:
# Overview of all datasets
print("=" * 80)
print("ALL DATASETS OVERVIEW")
print("=" * 80)
for name, df in dataframes.items():
    print(f"\n{name}:")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

ALL DATASETS OVERVIEW

airline:
  Shape: 27,284 rows × 15 columns
  Columns: ['airline_name', 'link', 'title', 'author', 'author_country', 'date', 'content', 'cabin_flown', 'overall_rating', 'seat_comfort_rating', 'cabin_staff_rating', 'food_beverages_rating', 'inflight_entertainment_rating', 'value_money_rating', 'recommended']
  Memory: 32.58 MB

airport:
  Shape: 10,817 rows × 12 columns
  Columns: ['airport_name', 'link', 'title', 'author', 'author_country', 'date', 'content', 'overall_rating', 'queuing_rating', 'terminal_cleanliness_rating', 'airport_shopping_rating', 'recommended']
  Memory: 11.80 MB

lounge:
  Shape: 1,470 rows × 19 columns
  Columns: ['airline_name', 'link', 'title', 'author', 'author_country', 'date', 'content', 'lounge_name', 'airport', 'lounge_type', 'overall_rating', 'comfort_rating', 'cleanliness_rating', 'bar_beverages_rating', 'catering_rating', 'washrooms_rating', 'wifi_connectivity_rating', 'staff_service_rating', 'recommended']
  Memory: 1.76 MB

seat

### 1. Basic Dataset Properties for All Datasets

In [24]:
# Detailed properties for each dataset
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name}")
    print("=" * 80)
    print(f"\nTotal Records: {df.shape[0]:,}")
    print(f"Total Columns: {df.shape[1]}")
    print(f"\nColumn Names and Data Types:")
    print("-" * 80)
    for col, dtype in df.dtypes.items():
        null_count = df[col].isnull().sum()
        null_pct = (null_count / len(df)) * 100
        print(f"{col:30} | {str(dtype):15} | Nulls: {null_count:7,} ({null_pct:5.2f}%)")
    print("\n")

DATASET: airline

Total Records: 27,284
Total Columns: 15

Column Names and Data Types:
--------------------------------------------------------------------------------
airline_name                   | object          | Nulls:       0 ( 0.00%)
link                           | object          | Nulls:       0 ( 0.00%)
title                          | object          | Nulls:       0 ( 0.00%)
author                         | object          | Nulls:       0 ( 0.00%)
author_country                 | object          | Nulls:       0 ( 0.00%)
date                           | object          | Nulls:       0 ( 0.00%)
content                        | object          | Nulls:       0 ( 0.00%)
cabin_flown                    | object          | Nulls:       0 ( 0.00%)
overall_rating                 | float64         | Nulls:       0 ( 0.00%)
seat_comfort_rating            | float64         | Nulls:       0 ( 0.00%)
cabin_staff_rating             | float64         | Nulls:       0 ( 0.00%)
food_b

### 2. Distribution Analysis Across All Datasets

In [25]:
# Distribution by airline (if applicable)
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - AIRLINE DISTRIBUTION")
    print("=" * 80)
    
    airline_col = [col for col in df.columns if 'airline' in col.lower()]
    if airline_col:
        airline_counts = df[airline_col[0]].value_counts()
        print(f"\nTotal Airlines: {airline_counts.shape[0]:,}")
        print(f"\nTop 10 Airlines by Count:")
        print("-" * 80)
        display(airline_counts.head(10))
    else:
        print("No airline column found in this dataset")
    print("\n")

DATASET: airline - AIRLINE DISTRIBUTION

Total Airlines: 292

Top 10 Airlines by Count:
--------------------------------------------------------------------------------


airline_name
british-airways           855
united-airlines           803
air-canada-rouge          703
emirates                  685
lufthansa                 586
american-airlines         579
qantas-airways            576
etihad-airways            512
qatar-airways             491
cathay-pacific-airways    491
Name: count, dtype: int64



DATASET: airport - AIRLINE DISTRIBUTION
No airline column found in this dataset


DATASET: lounge - AIRLINE DISTRIBUTION

Total Airlines: 85

Top 10 Airlines by Count:
--------------------------------------------------------------------------------


airline_name
british-airways      175
emirates             101
united-airlines       91
qantas-airways        86
malaysia-airlines     59
lufthansa             58
qatar-airways         52
thai-airways          48
air-france            47
air-canada            41
Name: count, dtype: int64



DATASET: seat - AIRLINE DISTRIBUTION

Total Airlines: 90

Top 10 Airlines by Count:
--------------------------------------------------------------------------------


airline_name
emirates                   84
british-airways            82
cathay-pacific-airways     76
virgin-atlantic-airways    75
lufthansa                  66
air-france                 64
qantas-airways             64
singapore-airlines         53
etihad-airways             46
american-airlines          45
Name: count, dtype: int64

In [26]:
# Distribution by ratings/scores
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - RATING/SCORE DISTRIBUTION")
    print("=" * 80)
    
    rating_col = [col for col in df.columns if 'rating' in col.lower() or 'score' in col.lower() or 'stars' in col.lower()]
    if rating_col:
        for col in rating_col[:3]:  # Show first 3 rating columns
            print(f"\n{col}:")
            print("-" * 40)
            display(df[col].value_counts().sort_index())
    else:
        print("No rating/score columns found in this dataset")
    print("\n")

DATASET: airline - RATING/SCORE DISTRIBUTION

overall_rating:
----------------------------------------


overall_rating
1.0     3906
2.0     2271
3.0     1782
4.0     1227
5.0     1885
6.0     1369
7.0     2451
8.0     3932
9.0     4008
10.0    4453
Name: count, dtype: int64


seat_comfort_rating:
----------------------------------------


seat_comfort_rating
0.0     279
1.0    4076
2.0    3163
3.0    5769
4.0    8760
5.0    5237
Name: count, dtype: int64


cabin_staff_rating:
----------------------------------------


...

overall_rating
1.0     347
2.0     143
3.0     111
4.0      89
5.0      80
6.0      66
7.0      91
8.0     139
9.0     100
10.0     43
Name: count, dtype: int64


seat_legroom_rating:
----------------------------------------


seat_legroom_rating
1    356
2    204
3    223
4    246
5    180
Name: count, dtype: int64


seat_recline_rating:
----------------------------------------


seat_recline_rating
1    310
2    245
3    308
4    259
5     87
Name: count, dtype: int64

In [27]:
# Distribution by date/time
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - DATE DISTRIBUTION")
    print("=" * 80)
    
    date_col = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower() or 'year' in col.lower()]
    if date_col:
        df_temp = df.copy()
        df_temp[date_col[0]] = pd.to_datetime(df_temp[date_col[0]], errors='coerce')
        
        valid_dates = df_temp[date_col[0]].dropna()
        if len(valid_dates) > 0:
            print(f"\nDate Range: {valid_dates.min()} to {valid_dates.max()}")
            print(f"\nRecords by Year:")
            print("-" * 40)
            yearly = valid_dates.dt.year.value_counts().sort_index()
            display(yearly)
        else:
            print("Could not parse dates")
    else:
        print("No date columns found in this dataset")
    print("\n")

DATASET: airline - DATE DISTRIBUTION

Date Range: 1970-01-01 00:00:00 to 2015-08-02 00:00:00

Records by Year:
----------------------------------------


date
1970        1
2011      325
2012     3136
2013     6766
2014    11388
2015     5668
Name: count, dtype: int64



DATASET: airport - DATE DISTRIBUTION

Date Range: 2008-09-28 00:00:00 to 2015-08-01 00:00:00

Records by Year:
----------------------------------------


date
2008       1
2009     355
2010    1143
2011    1956
2012    2453
2013    1904
2014    1819
2015    1186
Name: count, dtype: int64



DATASET: lounge - DATE DISTRIBUTION

Date Range: 2007-11-02 00:00:00 to 2015-08-02 00:00:00

Records by Year:
----------------------------------------


date
2007      1
2008      1
2009      2
2010     12
2011    200
2012    280
2013    352
2014    346
2015    276
Name: count, dtype: int64



DATASET: seat - DATE DISTRIBUTION

Date Range: 1970-01-01 00:00:00 to 2015-08-02 00:00:00

Records by Year:
----------------------------------------


date
1970     25
2007      1
2008     57
2009    107
2010    110
2011    159
2012     23
2013    150
2014    323
2015    254
Name: count, dtype: int64

In [28]:
# Distribution by routes/locations
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - ROUTE/LOCATION DISTRIBUTION")
    print("=" * 80)
    
    route_col = [col for col in df.columns if any(keyword in col.lower() for keyword in ['route', 'origin', 'destination', 'country', 'location'])]
    if route_col:
        for col in route_col[:2]:  # Show first 2 route-related columns
            print(f"\n{col}:")
            print("-" * 40)
            print(f"Unique values: {df[col].nunique():,}")
            print(f"\nTop 10 most common:")
            display(df[col].value_counts().head(10))
    else:
        print("No route/location columns found in this dataset")
    print("\n")

DATASET: airline - ROUTE/LOCATION DISTRIBUTION

author_country:
----------------------------------------
Unique values: 142

Top 10 most common:


author_country
United Kingdom    6275
United States     4967
Australia         3931
Canada            2625
Germany            885
Singapore          528
New Zealand        489
India              431
Netherlands        364
France             350
Name: count, dtype: int64



DATASET: airport - ROUTE/LOCATION DISTRIBUTION

author_country:
----------------------------------------
Unique values: 112

Top 10 most common:


author_country
United Kingdom    4040
United States     1759
Australia          997
Canada             451
Germany            234
Netherlands        180
France             179
Singapore          171
New Zealand        162
India              159
Name: count, dtype: int64



DATASET: lounge - ROUTE/LOCATION DISTRIBUTION

author_country:
----------------------------------------
Unique values: 64

Top 10 most common:


author_country
United Kingdom    392
Australia         279
United States     222
Canada             74
New Zealand        34
Brazil             30
Germany            30
Hong Kong          26
France             23
Thailand           23
Name: count, dtype: int64



DATASET: seat - ROUTE/LOCATION DISTRIBUTION

author_country:
----------------------------------------
Unique values: 58

Top 10 most common:


author_country
United Kingdom    333
United States     226
Australia         171
Canada             60
New Zealand        44
Germany            44
Singapore          34
Hong Kong          23
South Africa       18
Netherlands        17
Name: count, dtype: int64

### 3. Text Length Statistics Across Datasets

In [29]:
# Text length analysis for all datasets
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - TEXT LENGTH STATISTICS")
    print("=" * 80)
    
    text_col = [col for col in df.columns if any(keyword in col.lower() for keyword in ['review', 'text', 'comment', 'content', 'description'])]
    
    if text_col:
        for col in text_col[:2]:  # Analyze first 2 text columns
            print(f"\n{col}:")
            print("-" * 80)
            text_lengths = df[col].dropna().astype(str).str.len()
            word_counts = df[col].dropna().astype(str).str.split().str.len()
            
            if len(text_lengths) > 0:
                print(f"Non-null entries: {len(text_lengths):,}")
                print(f"Minimum length: {text_lengths.min()} characters")
                print(f"Maximum length: {text_lengths.max()} characters")
                print(f"Average length: {text_lengths.mean():.2f} characters")
                print(f"Median length: {text_lengths.median():.2f} characters")
                print(f"\nMinimum words: {word_counts.min()}")
                print(f"Maximum words: {word_counts.max()}")
                print(f"Average words: {word_counts.mean():.2f}")
                print(f"Median words: {word_counts.median():.2f}")
                
                print(f"\nLength Distribution (percentiles):")
                display(text_lengths.describe())
    else:
        print("No text columns found in this dataset")
    print("\n")

DATASET: airline - TEXT LENGTH STATISTICS

content:
--------------------------------------------------------------------------------
Non-null entries: 27,284
Minimum length: 77 characters
Maximum length: 3610 characters
Average length: 664.94 characters
Median length: 569.00 characters

Minimum words: 12
Maximum words: 652
Average words: 119.52
Median words: 102.00

Length Distribution (percentiles):
Non-null entries: 27,284
Minimum length: 77 characters
Maximum length: 3610 characters
Average length: 664.94 characters
Median length: 569.00 characters

Minimum words: 12
Maximum words: 652
Average words: 119.52
Median words: 102.00

Length Distribution (percentiles):


count    27284.000000
mean       664.940698
std        407.532739
min         77.000000
25%        380.000000
50%        569.000000
75%        836.000000
max       3610.000000
Name: content, dtype: float64



DATASET: airport - TEXT LENGTH STATISTICS

content:
--------------------------------------------------------------------------------
Non-null entries: 10,817
Minimum length: 52 characters
Maximum length: 4449 characters
Average length: 621.86 characters
Median length: 535.00 characters

Minimum words: 9
Maximum words: 822
Average words: 111.11
Median words: 95.00

Length Distribution (percentiles):


count    10817.000000
mean       621.856799
std        383.428463
min         52.000000
25%        353.000000
50%        535.000000
75%        790.000000
max       4449.000000
Name: content, dtype: float64



DATASET: lounge - TEXT LENGTH STATISTICS

content:
--------------------------------------------------------------------------------
Non-null entries: 1,470
Minimum length: 91 characters
Maximum length: 2190 characters
Average length: 464.41 characters
Median length: 387.50 characters

Minimum words: 13
Maximum words: 414
Average words: 83.02
Median words: 69.00

Length Distribution (percentiles):


count    1470.000000
mean      464.412925
std       292.285397
min        91.000000
25%       262.000000
50%       387.500000
75%       591.000000
max      2190.000000
Name: content, dtype: float64



DATASET: seat - TEXT LENGTH STATISTICS

content:
--------------------------------------------------------------------------------
Non-null entries: 1,209
Minimum length: 32 characters
Maximum length: 3141 characters
Average length: 433.96 characters
Median length: 349.00 characters

Minimum words: 7
Maximum words: 594
Average words: 80.49
Median words: 64.00

Length Distribution (percentiles):


count    1209.000000
mean      433.960298
std       311.174822
min        32.000000
25%       223.000000
50%       349.000000
75%       538.000000
max      3141.000000
Name: content, dtype: float64

### 4. Missing Data Patterns Across All Datasets

In [30]:
# Missing data analysis for all datasets
for name, df in dataframes.items():
    print("=" * 80)
    print(f"DATASET: {name} - MISSING DATA PATTERNS")
    print("=" * 80)
    
    missing_data = df.isnull().sum()
    missing_pct = (missing_data / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing_data.index,
        'Missing_Count': missing_data.values,
        'Missing_Percentage': missing_pct.values
    }).sort_values('Missing_Count', ascending=False)
    
    print("\nColumns with Missing Data:")
    print("-" * 80)
    missing_cols = missing_df[missing_df['Missing_Count'] > 0]
    if len(missing_cols) > 0:
        display(missing_cols)
    else:
        print("No missing values in this dataset!")
    print("\n")

DATASET: airline - MISSING DATA PATTERNS

Columns with Missing Data:
--------------------------------------------------------------------------------
No missing values in this dataset!


DATASET: airport - MISSING DATA PATTERNS

Columns with Missing Data:
--------------------------------------------------------------------------------
No missing values in this dataset!


DATASET: lounge - MISSING DATA PATTERNS

Columns with Missing Data:
--------------------------------------------------------------------------------
No missing values in this dataset!


DATASET: seat - MISSING DATA PATTERNS

Columns with Missing Data:
--------------------------------------------------------------------------------
No missing values in this dataset!



Columns with Missing Data:
--------------------------------------------------------------------------------
No missing values in this dataset!


DATASET: lounge - MISSING DATA PATTERNS

Columns with Missing Data:
------------------------------------------

### 5. Potential Computational Problems

Based on all dataset properties, let's identify computational challenges for analysis and algorithms.

In [31]:
# Identify computational problems across all datasets
print("=" * 80)
print("POTENTIAL COMPUTATIONAL PROBLEMS - COMPREHENSIVE ANALYSIS")
print("=" * 80)

all_problems = {}

for name, df in dataframes.items():
    problems = []
    
    # Problem 1: Dataset size
    total_memory = df.memory_usage(deep=True).sum() / (1024**2)
    if df.shape[0] > 50000 or total_memory > 50:
        problems.append({
            'Problem': 'Large Dataset Size',
            'Details': f'{df.shape[0]:,} records, {total_memory:.2f} MB memory',
            'Impact': 'Memory-intensive operations, slow processing for complex algorithms',
            'Considerations': 'Chunking, sampling, or distributed processing may be needed'
        })
    
    # Problem 2: Imbalanced distribution
    airline_col = [col for col in df.columns if 'airline' in col.lower()]
    if airline_col:
        airline_counts = df[airline_col[0]].value_counts()
        if len(airline_counts) > 0:
            max_count = airline_counts.max()
            min_count = airline_counts.min()
            ratio = max_count / min_count if min_count > 0 else float('inf')
            if ratio > 50:
                problems.append({
                    'Problem': 'Imbalanced Distribution',
                    'Details': f'Max: {max_count:,}, Min: {min_count:,}, Ratio: {ratio:.1f}:1',
                    'Impact': 'Bias in rankings and analysis towards popular entities',
                    'Considerations': 'Normalization, weighted scoring, or minimum threshold filtering'
                })
    
    # Problem 3: Text processing complexity
    text_col = [col for col in df.columns if any(keyword in col.lower() for keyword in ['review', 'text', 'comment', 'content'])]
    if text_col:
        text_lengths = df[text_col[0]].dropna().astype(str).str.len()
        if len(text_lengths) > 0:
            total_chars = text_lengths.sum()
            if total_chars > 5_000_000:  # 5M characters
                problems.append({
                    'Problem': 'Large Text Corpus',
                    'Details': f'{total_chars:,} chars, avg {text_lengths.mean():.0f} chars/entry',
                    'Impact': 'NLP operations (sentiment, embeddings) will be time-consuming',
                    'Considerations': 'Pre-processing pipeline, caching, efficient libraries (spaCy, transformers)'
                })
    
    # Problem 4: High cardinality
    high_cardinality_cols = []
    for col in df.select_dtypes(include=['object']).columns:
        unique_count = df[col].nunique()
        if unique_count > 500:
            high_cardinality_cols.append((col, unique_count))
    
    if high_cardinality_cols:
        details = ', '.join([f'{col}: {count:,}' for col, count in high_cardinality_cols])
        problems.append({
            'Problem': 'High Cardinality Features',
            'Details': details,
            'Impact': 'Explosion of features in ML models, memory issues',
            'Considerations': 'Feature hashing, embeddings, dimensionality reduction, or grouping'
        })
    
    # Problem 5: Missing data
    missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    if missing_pct > 10:
        problems.append({
            'Problem': 'Significant Missing Data',
            'Details': f'{missing_pct:.2f}% of all values are missing',
            'Impact': 'Reduced data quality, potential bias in analysis',
            'Considerations': 'Imputation strategies, or filtering rows/columns with too many nulls'
        })
    
    all_problems[name] = problems

# Display problems by dataset
for name, problems in all_problems.items():
    print(f"\n{'='*80}")
    print(f"DATASET: {name}")
    print('='*80)
    
    if problems:
        for i, problem in enumerate(problems, 1):
            print(f"\n{i}. {problem['Problem']}")
            print("-" * 80)
            print(f"Details: {problem['Details']}")
            print(f"Impact: {problem['Impact']}")
            print(f"Considerations: {problem['Considerations']}")
    else:
        print("\nNo major computational problems identified for this dataset.")
    print()

# Summary
total_problems = sum(len(p) for p in all_problems.values())
print(f"\n{'='*80}")
print(f"SUMMARY: {total_problems} total computational challenges identified across all datasets")
print('='*80)

POTENTIAL COMPUTATIONAL PROBLEMS - COMPREHENSIVE ANALYSIS

DATASET: airline

1. Imbalanced Distribution
--------------------------------------------------------------------------------
Details: Max: 855, Min: 1, Ratio: 855.0:1
Impact: Bias in rankings and analysis towards popular entities
Considerations: Normalization, weighted scoring, or minimum threshold filtering

2. Large Text Corpus
--------------------------------------------------------------------------------
Details: 18,142,242 chars, avg 665 chars/entry
Impact: NLP operations (sentiment, embeddings) will be time-consuming
Considerations: Pre-processing pipeline, caching, efficient libraries (spaCy, transformers)

3. High Cardinality Features
--------------------------------------------------------------------------------
Details: author: 19,634, date: 956, content: 27,265
Impact: Explosion of features in ML models, memory issues
Considerations: Feature hashing, embeddings, dimensionality reduction, or grouping


DATASET: air

## Export Cleaned Data

Now that we've cleaned the datasets, let's export them so they can be used in other files for data structure implementations and analysis.

In [32]:
# Create a cleaned_data directory to store processed datasets
cleaned_data_dir = Path('../data/cleaned')
cleaned_data_dir.mkdir(parents=True, exist_ok=True)

print("Exporting cleaned datasets...")
print("=" * 80)

for name, df in dataframes.items():
    output_file = cleaned_data_dir / f"{name}_cleaned.csv"
    df.to_csv(output_file, index=False)
    print(f"✓ Exported {name}: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"  → {output_file}")

print("\n" + "=" * 80)
print(f"All cleaned datasets saved to: {cleaned_data_dir.absolute()}")
print("=" * 80)

Exporting cleaned datasets...
✓ Exported airline: 27,284 rows × 15 columns
  → ..\data\cleaned\airline_cleaned.csv
✓ Exported airport: 10,817 rows × 12 columns
  → ..\data\cleaned\airport_cleaned.csv
✓ Exported lounge: 1,470 rows × 19 columns
  → ..\data\cleaned\lounge_cleaned.csv
✓ Exported seat: 1,209 rows × 17 columns
  → ..\data\cleaned\seat_cleaned.csv

All cleaned datasets saved to: c:\Users\easan.s.2024\Documents\GitHub\CS201-G4T6\TopK-Airlines\EDA\..\data\cleaned


In [33]:
# Also save a summary of the cleaned datasets
summary_file = cleaned_data_dir / 'cleaning_summary.txt'

with open(summary_file, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("DATA CLEANING SUMMARY\n")
    f.write("=" * 80 + "\n\n")
    
    for name, df in dataframes.items():
        f.write(f"Dataset: {name}\n")
        f.write("-" * 80 + "\n")
        f.write(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns\n")
        f.write(f"Columns: {', '.join(df.columns)}\n")
        f.write(f"Memory: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB\n")
        f.write(f"Missing values: {df.isnull().sum().sum()}\n")
        f.write("\n")

print(f"Summary saved to: {summary_file}")

Summary saved to: ..\data\cleaned\cleaning_summary.txt


## Using Cleaned Data in Other Files

The cleaned datasets are now ready to use! Here's how to load them in your Python scripts:

```python
# In your Python scripts (e.g., in experiments/ or src/)
from src.utils.data_loader import load_cleaned_data

# Option 1: Load all cleaned datasets
all_data = load_cleaned_data()
airline_df = all_data['airline']
airport_df = all_data['airport']
lounge_df = all_data['lounge']
seat_df = all_data['seat']

# Option 2: Load a specific dataset
airline_df = load_cleaned_data('airline')

# Now you can use the dataframes with your data structures!
```

See `examples/load_cleaned_data_example.py` for complete examples of how to:
- Load and access the cleaned data
- Prepare data for heaps and priority queues
- Create hash table mappings
- Filter and prepare data for Top-K algorithms